In [12]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    f"""Extract the desired information from the following passage.
    only extract the properties mentioned in the 'Classification' function.
    Passage: {input}
    """
)


class Classification(BaseModel):
    sentiment: str = Field(description="Sentiment of the text.")
    aggressiveness: int = Field(description="Aggressiveness of the text.")
    language: str = Field(description="Language of the text.")


structured_llm = llm.with_structured_output(Classification)

inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

# response
response.model_dump()

{'sentiment': 'neutral', 'aggressiveness': 0, 'language': 'en'}

In [27]:
from typing import Literal


class Classification(BaseModel):
    sentiment: Literal['happy', 'neutral', 'sad', 'angry'] = Field(..., description="Sentiment of the text.")
    aggressiveness: Literal[1, 2, 3, 4, 5] = Field(..., description="Aggressiveness of the text.")
    language: Literal["spanish", "english", "french", "german", "italian"] = Field(...,
                                                                                   description="Language of the text.")
    behaviour: Literal['Good Behave', 'Bad Behave'] = Field(description="Whether the text is good or bad.")


In [29]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

- sentiment: happy, neutral, sad, angry
- aggressiveness: 1 is low to 5 is high
- language: spanish, english, french, german, italian
- behaviour: Good Behave, Bad Behave

Rules:
1. Detect the language carefully; do not assume English by default.
2. Treat threats or insults as high aggressiveness and Bad Behave.
3. Sentiment should reflect the overall emotion in the text.
4. Always output exactly the fields in the Classification model.

Passage:
{input}
"""
)


llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='spanish', behaviour='Good Behave')

In [30]:
inp = "I will kill you, if you will not come"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='angry', aggressiveness=5, language='english', behaviour='Bad Behave')